In [3]:
import pyTigerGraph as tg
import pandas as pd
import datetime
import pysftp
import cfg


In [4]:
cfg.token = tg.TigerGraphConnection(host="http://1.186.48.107/", graphname="anm_real_graph").getToken(cfg.secret)[0]
conn = tg.TigerGraphConnection(host="http://1.186.48.107/", graphname="anm_real_graph", username="tigergraph",password=cfg.password,apiToken=cfg.token)

In [6]:
Hostname = "162.27.169.18"
Username = "AlvrzMrsl_LOG001"
Password = "{x~[R}5x"

In [7]:
try:
  with pysftp.Connection(host=Hostname, username=Username, password=Password,port=22) as sftp:
    print("Connection successfully established ... ")
    localFilePath_stop = 'SFTP_Data/stop.csv'
    localFilePath_mfst = 'SFTP_Data/mfst.csv'
    remoteFilePath_stop = '/PROD/From_LSC/2205162245_stop.csv'
    remoteFilePath_mfst = '/PROD/From_LSC/2205162245_mfst.csv'
    sftp.get (remoteFilePath_stop,localFilePath_stop)
    sftp.get (remoteFilePath_mfst,localFilePath_mfst)
except:
  print('failed to establish connection to targeted server')
sftp.close()

Connection successfully established ... 


In [8]:
def to_list(row):
    try:
        return row.split(",")
    except:
        return []
def cal_epoch(val):
    try:
        dt_obj = datetime.datetime.strptime(val, '%m/%d/%Y %H:%M')  
        epoch_sec = int(dt_obj.timestamp())
        return epoch_sec
    except:
        return "NA"

In [9]:

stop_df = pd.read_csv(localFilePath_stop)
mfst_df = pd.read_csv(localFilePath_mfst)
mfst_df['Status'] = mfst_df['Status'].apply(to_list)
mfst_df

,Manifest,Status,Origin Lane,Dest Lane,Origin-Dest Lane,RTB Date,RTB Time,Weight
0,54146057,[U],VA228,WI549,ROCKINGHAM VA-APPLETON WI,05/16/2022,10:35,0
1,106483810,[U],MD217,IN461,HAGERSTOWN MD-PLAINFIELD IN,05/17/2022,10:00,335
2,106494201,[U],VA228,NJ088,ROCKINGHAM VA-EAST BRUNSWICK NJ,05/17/2022,08:00,50
3,106494383,[U],VA228,GA306,ROCKINGHAM VA-WINDER GA,05/17/2022,05:00,130
4,106494395,[U],PA166,IL604,WILLIAMSBURG PA-PALOS HILLS IL,05/17/2022,09:00,290
5,106494644,[U],WI541,IA503,DE PERE WI-DES MOINES IA,05/18/2022,14:00,0
6,106494649,[U],VA228,IL604,ROCKINGHAM VA-ROMEOVILLE IL,05/17/2022,18:00,150


In [10]:
stop_df['Stop Seq']=stop_df['Stop Seq'].astype(str)
stop_df['Manifest'] = stop_df['Manifest'].astype(str)
stop_df['Stop Location#'] = stop_df['Stop Location#'].astype(str)
stop_df['Stop Type'] = stop_df['Stop Type'].apply(to_list)
stop_df['id'] = stop_df['Manifest']+"-" + stop_df['Stop Seq']+"-"+stop_df['Stop Location#']
stop_df['inter_date'] = stop_df['Scheduled Date']+ " "+stop_df['Scheduled Time']
stop_df['epoch_sch_start'] = stop_df['inter_date'].apply(cal_epoch).astype('int64')
stop_df['epoch_sch_end'] = stop_df['epoch_sch_start'].astype('int64') + 7200
stop_df['must_start'] = stop_df['epoch_sch_start'].astype('int64') - 3600
stop_df['must_end'] = stop_df['epoch_sch_end'].astype('int64') + 3600
stop_df['load_time']=7200
stop_df

,Manifest,Stop Seq,Stop Type,Stop Location#,Scheduled Date,Scheduled Time,id,inter_date,epoch_sch_start,epoch_sch_end,must_start,must_end,load_time
0,54146057,1,[O],12,05/20/2022,11:00,54146057-1-12,05/20/2022 11:00,1653024600,1653031800,1653021000,1653035400,7200
1,54146057,2,[S],165337,05/23/2022,18:00,54146057-2-165337,05/23/2022 18:00,1653309000,1653316200,1653305400,1653319800,7200
2,106483810,1,[O],102316,05/17/2022,10:00,106483810-1-102316,05/17/2022 10:00,1652761800,1652769000,1652758200,1652772600,7200
3,106483810,2,[S],375554,05/18/2022,12:00,106483810-2-375554,05/18/2022 12:00,1652855400,1652862600,1652851800,1652866200,7200
4,106494201,1,[O],12,05/17/2022,08:00,106494201-1-12,05/17/2022 08:00,1652754600,1652761800,1652751000,1652765400,7200
5,106494201,2,[S],146041,05/18/2022,08:00,106494201-2-146041,05/18/2022 08:00,1652841000,1652848200,1652837400,1652851800,7200
6,106494383,1,[O],12,05/17/2022,15:00,106494383-1-12,05/17/2022 15:00,1652779800,1652787000,1652776200,1652790600,7200
7,106494383,2,[S],342617,05/18/2022,11:00,106494383-2-342617,05/18/2022 11:00,1652851800,1652859000,1652848200,1652862600,7200
8,106494395,1,[O],354954,05/17/2022,14:30,106494395-1-354954,05/17/2022 14:30,1652778000,1652785200,1652774400,1652788800,7200
9,106494395,2,[S],7784,05/18/2022,09:00,106494395-2-7784,05/18/2022 09:00,1652844600,1652851800,1652841000,1652855400,7200


In [20]:
stop_df.sort_values('id')
g_event_df = stop_df.groupby('Manifest')
g_event_df.first()

,Stop Seq,Stop Type,Stop Location#,Scheduled Date,Scheduled Time,id,inter_date,epoch_sch_start,epoch_sch_end,must_start,must_end,load_time
Manifest,,,,,,,,,,,,
106483810,1,[O],102316,05/17/2022,10:00,106483810-1-102316,05/17/2022 10:00,1652761800,1652769000,1652758200,1652772600,7200
106494201,1,[O],12,05/17/2022,08:00,106494201-1-12,05/17/2022 08:00,1652754600,1652761800,1652751000,1652765400,7200
106494383,1,[O],12,05/17/2022,15:00,106494383-1-12,05/17/2022 15:00,1652779800,1652787000,1652776200,1652790600,7200
106494395,1,[O],354954,05/17/2022,14:30,106494395-1-354954,05/17/2022 14:30,1652778000,1652785200,1652774400,1652788800,7200
106494644,1,[O],103704,05/18/2022,14:00,106494644-1-103704,05/18/2022 14:00,1652862600,1652869800,1652859000,1652873400,7200
106494649,1,[O],12,05/17/2022,18:00,106494649-1-12,05/17/2022 18:00,1652790600,1652797800,1652787000,1652801400,7200
54146057,1,[O],12,05/20/2022,11:00,54146057-1-12,05/20/2022 11:00,1653024600,1653031800,1653021000,1653035400,7200


In [18]:
source = []
target = []
for group in g_event_df.groups:
    temp_df = g_event_df.get_group(group)
    for i in range(len(temp_df)-1):
        source.append(temp_df['id'].iloc[i])
        target.append(temp_df['id'].iloc[i+1])

data = {
    "Source": source,
    "target": target,
    "required": True
}

followby_df = pd.DataFrame(data)

followby_df.head()

,Source,target,required
0,106483810-1-102316,106483810-2-375554,True
1,106494201-1-12,106494201-2-146041,True
2,106494383-1-12,106494383-2-342617,True
3,106494395-1-354954,106494395-2-7784,True
4,106494644-1-103704,106494644-2-126792,True


In [84]:
conn.upsertVertexDataFrame(mfst_df,"bill_of_events","Manifest",{
    "weight":"Weight",
    "bil_type":"Status"
})

7

In [85]:
conn.upsertVertexDataFrame(stop_df,"event","id",{
    "event_type":"Stop Type",
    "plan_start":"epoch_sch_start",
    "plan_end": "epoch_sch_end" ,
    "must_start_by":"must_start",
    "must_end_by" :"must_end",
    "load_time":"load_time"
})

14

In [86]:
conn.upsertEdgeDataFrame(followby_df,"event","follow_by","event","Source","target",{
    "required" : "required"
})

7

In [89]:
conn.upsertEdgeDataFrame(stop_df,"location","happens_at","event","Stop Location#","id",{})

14

In [88]:
conn.upsertEdgeDataFrame(stop_df,"event","belongs_to","bill_of_events","id","Manifest",{
    "belongs_to_sequence" : "Stop Seq"
})

14